In [1]:
import numpy as np
import pandas as pd
import jieba as jb
import matplotlib.pyplot as plt
%matplotlib inline
import math
import time
import datetime
import os
import re
import io
import csv
import shutil
import sys
import mysql.connector as sql
from mysql.connector import MySQLConnection, Error
default_stdout = sys.stdout
default_stderr = sys.stderr
reload(sys)
sys.stdout = default_stdout
sys.stderr = default_stderr
sys.setdefaultencoding('utf-8')

In [2]:
weekday_time_list = []
for dd in range(7):
    for tt in range(24):
        weekday_time_list.append((dd,tt))
        
weekday_time_list=pd.DataFrame(weekday_time_list,columns=["weekday",'time'])    
        

In [3]:
# 合成一個最終table是給web呈現
xls='vd_real_info.xlsx'
vd_info_excel_df=pd.read_excel(xls,encoding='utf-8');
vd_info_excel_dfc=vd_info_excel_df.copy()

pattern= {'direct':{u'.?東$':'東',u'.?西$':'西',u'.?南$':'南',u'.?北$':'北',u'.向放置':'未指定'}\
          ,'newdirect':{u'.?東.?':'東',u'.?西.?':'西',u'.?南.?':'南',u'.?北.?':'北',u'.向放置':'未指定'}}

vd_info_excel_dfc=vd_info_excel_dfc.replace(pattern,regex=True);

pdbc = sql.connect(host='localhost', database='taipeiall', user='root', password='apple')
db_table_df=pd.read_sql('show tables;', con=pdbc)
db_table_dfc=db_table_df.copy()
db_table_dfc.columns=['tablename']
db_table_dfc['vd'] = db_table_dfc.ix[:,0].str[1:]
db_table_dfc['direct'] = db_table_dfc.ix[:,0].str[:1]
pattern= {'direct':{u'E':'東',u'W':'西',u'S':'南',u'N':'北',u'U':'未指定'}}
db_table_dfc=db_table_dfc.replace(pattern,regex=True);


mergetb = pd.merge(vd_info_excel_dfc,db_table_dfc,on=['vd','direct'])


In [4]:
mergetb.sortby=mergetb.index

In [5]:

mergetb=mergetb.assign(accu =0,speed=0,jampercent=0,congest= 0)

In [6]:
frames= []
sortlist=mergetb.sortby.tolist()
for i in sortlist:
    temp=weekday_time_list.assign(sortby=i)
    temp2=mergetb.ix[mergetb.sortby==i]
    frames.append(pd.merge(temp2,temp,on=["sortby"]))
web_table = pd.concat(frames,ignore_index=True)

In [7]:
# 假裝有預測值了！！！把預測的速度跟佔有率塞進去
web_table['speed']=pd.DataFrame({"speed":np.random.rand(len(web_table.jampercent))*80})
web_table['accu']=pd.DataFrame({"accu":np.random.rand(len(web_table.jampercent))*25})
# web_table


In [9]:
# dbtable=db_table_df.ix[:,0].tolist()
# query = []
# for table in dbtable:
#     query.append("select * from {}".format(table))
# query_alltable=" union ".join(query)+";"

In [11]:
def decode_to_utf8(df):
    df.EQIPnumber=df.EQIPnumber.str.decode("utf-8")
    df.location=df.location.str.decode("utf-8")
    df.direct=df.direct.str.decode("utf-8")
    df.YMD=df.YMD.str.decode("utf-8")
    df.hour=df.hour.str.decode("utf-8")
    return df

pdbc = sql.connect(host='localhost', database='taipeiall', user='root', password='apple')

dbtable=db_table_df.ix[:,0].tolist()
import time
tStart = time.time()
frames = []
for ele in dbtable:
    frames.append(pd.read_sql('SELECT * FROM {r}'.format(r=ele), con=pdbc))

result = pd.concat(frames, ignore_index=True)

df_all=decode_to_utf8(result)

tEnd = time.time()
print "It cost %f sec" % (tEnd - tStart)


It cost 599.368596 sec


In [14]:
df_all=df_all.drop(['scooterflow','truckflow','addTotal','carflow','avgCarSpace','laneNumber'],axis=1)
hour_to_num={'00:00:00':0,'01:00:00':1,'02:00:00':2,'03:00:00':3,'04:00:00':4,'05:00:00':5,'06:00:00':6,'07:00:00':7,'08:00:00':8,'09:00:00':9,'10:00:00':10,'11:00:00':11,'12:00:00':12,'13:00:00':13,'14:00:00':14,'15:00:00':15,'16:00:00':16,'17:00:00':17,'18:00:00':18,'19:00:00':19,'20:00:00':20,'21:00:00':21,'22:00:00':22,'23:00:00':23}

pattern= {'direct':{u'.?東$':'東',u'.?西$':'西',u'.?南$':'南',u'.?北$':'北',u'.向放置':'未指定'},'hour':hour_to_num}
df_all=df_all.replace(pattern,regex=True);
df_all['YMD']=pd.to_datetime(df_all['YMD']);
df_all['weekday'] = df_all['YMD'].dt.dayofweek;
df_all.columns = ['vd','location','direct','YMD','time','eq','speed','accu','weekday']

In [15]:
# df_all.to_csv("table_all_0330.csv",index=False)

In [16]:
df_all

,vd,location,direct,YMD,time,eq,speed,accu,weekday
0,V0UVS20,指南路三段指南國小往南約60M處(附掛號誌燈桿),東,2016-04-01,0,11.694915,21.730000,0.27,4
1,V0UVS20,指南路三段指南國小往南約60M處(附掛號誌燈桿),東,2016-04-01,1,9.152542,29.109999,0.15,4
2,V0UVS20,指南路三段指南國小往南約60M處(附掛號誌燈桿),東,2016-04-01,2,4.067797,36.000000,0.07,4
3,V0UVS20,指南路三段指南國小往南約60M處(附掛號誌燈桿),東,2016-04-01,3,4.067797,37.000000,0.07,4
4,V0UVS20,指南路三段指南國小往南約60M處(附掛號誌燈桿),東,2016-04-01,4,8.135593,35.250000,0.13,4
5,V0UVS20,指南路三段指南國小往南約60M處(附掛號誌燈桿),東,2016-04-01,5,1.016949,37.000000,0.02,4
6,V0UVS20,指南路三段指南國小往南約60M處(附掛號誌燈桿),東,2016-04-01,6,1.016949,37.000000,0.02,4
7,V0UVS20,指南路三段指南國小往南約60M處(附掛號誌燈桿),東,2016-04-01,7,12.711864,37.330000,0.20,4
8,V0UVS20,指南路三段指南國小往南約60M處(附掛號誌燈桿),東,2016-04-01,8,39.827587,33.080000,0.69,4
9,V0UVS20,指南路三段指南國小往南約60M處(附掛號誌燈桿),東,2016-04-01,9,25.932203,31.200000,0.47,4
